In [2]:
import os
from datetime import date, datetime, timedelta

import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# 設定資料夾、城市和類別列表
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
city_list = ["TPE", "TYU", "TCH", "TNA", "KSH"]
type_list = ["salon", "restaurant", "supplies"]

# 先按照類別迴圈
for type_ in type_list:
    columns = ['name', 'place_id', 'buss_status', 'update_time']
    df_main = pd.DataFrame(columns=columns)

    # 依序將城市檔案讀入
    for city in city_list:
        file = f"gmap_{type_}_{city}.csv"
        path = os.path.join(folder, file)
        df_city = pd.read_csv(path)

        # 只保留正常營業的地標
        mask = df_city["buss_status"] == "OPERATIONAL"
        df_city = df_city[mask]

        # 根據place id去除重複
        df_city = df_city.drop_duplicates(subset=["place_id"], keep="first")
        print(f"完成{city}資料處理，共{len(df_city)}筆資料")

        # 跟主表合併
        df_main = pd.concat([df_main, df_city], ignore_index=True)

    # 合併完成再去重複一次
    df_main = df_main.drop_duplicates(subset=["place_id"], keep="first")

    # 重設索引（應該可以不用）
    # df_main = df_main.reset_index(drop=True)

    # 新增更新日期
    today = date.today().strftime('%Y/%m/%d')
    df_main["update_date"] = today

    # 存檔至地端
    combine_file = f"{type_}_place_id.csv"
    combine_path = os.path.join(folder, combine_file)
    df_main.to_csv(combine_path)
    print(f"已完成{type_}地端存檔")

    # 更新至資料庫
    username = os.getenv("MYSQL_USERNAME")
    password = os.getenv("MYSQL_PASSWORD")
    target_ip = os.getenv("MYSQL_IP")
    target_port = int(os.getenv("MYSQL_PORTT"))
    db_name = os.getenv("MYSQL_DB_NAME")

    engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

    df_main.to_sql(name=f"{type_}_place_id", con=engine, if_exists="replace", index=False)
    print(f"已完成{type_}資料庫存檔")

完成TPE資料處理，共646筆資料
完成TYU資料處理，共427筆資料
完成TCH資料處理，共425筆資料
完成TNA資料處理，共355筆資料
完成KSH資料處理，共404筆資料
已完成salon地端存檔
已完成salon資料庫存檔


In [ ]:
# # T_清理、去重複、重組資料

# columns = ['name', 'place_id', 'buss_status', 'update_time']
# main_df = pd.DataFrame(columns=columns)

# for city in city_list:
#     df_edit = df_dict[city].copy()

#     # 第一步篩選：將營業狀態非「正常營業」的地標去除
#     mask1 = df_edit["buss_status"] == "OPERATIONAL"
#     df_edit = df_edit[mask1]

#     # 第二部篩選：根據place id將重複的資料去除
#     df_edit = df_edit.drop_duplicates(subset=["place_id"], keep="first")
#     print(f"完成{city}資料處理，共{len(df_edit)}筆資料")

#     # 將城市資料合併至主要dataframe
#     main_df = pd.concat([main_df, df_edit])

# # 合併完成後再去重複一次
# main_df = main_df.drop_duplicates(subset=["place_id"], keep="first")

# # 重設索引
# main_df = main_df.reset_index(drop=True)

# # 新增id欄位並改變位置至最前
# main_df["id"] = "lodging_raw00" + main_df.index.astype(str)
# new_col = ["id", "name", "buss_status", "place_id"]
# main_df = main_df[new_col]
# print(f"完成所有資料整合，共{len(main_df)}筆資料")

# # 標記更新日期
# today = date.today().strftime('%Y/%m/%d')
# main_df["update_date"] = today

完成TPE資料處理，共318筆資料
完成TYU資料處理，共179筆資料
完成TCH資料處理，共223筆資料
完成TNA資料處理，共137筆資料
完成KSH資料處理，共169筆資料


In [ ]:
# # T_存檔為csv
# file_main = "gmap_lodging_main_placeid.csv"
# path_main = os.path.join(folder, file_main)

# main_df.to_csv(path_main, index=False)

In [ ]:
# # T_存檔至資料庫
# from sqlalchemy import create_engine
# from dotenv import load_dotenv

# load_dotenv()

# username = os.getenv("MYSQL_USERNAME")
# password = os.getenv("MYSQL_PASSWORD")
# target_ip = os.getenv("MYSQL_IP")
# target_port = int(os.getenv("MYSQL_PORTT"))
# db_name = os.getenv("MYSQL_DB_NAME")

# engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

# main_df.to_sql(name="lodging_placeid", con=engine, if_exists="replace", index=False)

978